# データ拡張

In [1]:
%load_ext autoreload
%autoreload 2

import os
import glob
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.applications import ResNet50

from modules.preprocessing import DataAugmentation
da = DataAugmentation

In [2]:
save_path = '/home/workspace/dataset/train/'
da = DataAugmentation(save_path)

In [ ]:
# main
ref_path = '/home/workspace/dataset/org/'
files = os.listdir(path)

c_num = [1,2,3,4]
data_num = [1,2,3,4,5,6,7,8,9,10]
for _, c in enumerate(c_num):
    for _, data in enumerate(data_num):
        filename = 'class{}_{}'.format(c,data)
        da.all_run(ref_path, filename, extension='.png')

# 献立推薦モデルの構築

## fine-tuningの準備

In [2]:
def create_model():
    # ベースモデル(ResNet)の構築
    base_model = ResNet50(include_top=False, weights='imagenet', pooling='avg')
    base_model.trainable = False
    # 出力層側に追加するレイヤーの構築
    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.Dense(4, activation="softmax")
    ], name='fine_tuning')
    return model

### 構築したモデルの確認

In [3]:
model = create_model()
model.summary()

94773248/94765736 [==============================] - 63s 1us/step
Model: "fine_tuning"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 4)                 8196      
Total params: 23,595,908
Trainable params: 23,542,788
Non-trainable params: 53,120
_________________________________________________________________


ここで，" Non-trainable params"が0出ない理由は，バッチ正則化層の重みがtrainable=Trueになっていても初期の段階ではFalseとして扱われるからである(多分)．なので，現段階で全ての重みが学習される状態となっている．

# 献立推薦モデルの学習

In [12]:
class_num = 4

In [ ]:
im_file_list = glob.glob(save_path+"*.png")
x_train = np.zeros([len(im_file_list),480,360,3])
for i, filename in enumerate(im_file_list):
    x_train[i] = np.array(Image.open(filename).resize((480,360))).transpose(1,0,2)

In [3]:
t_train = np.zeros([len(im_file_list),class_num])
labels = [[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]]
counter = 0
for _, label in enumerate(labels):
    for j in range(len(im_file_list)//class_num):
        t_train[counter] = label
        counter += 1

NameError: name 'im_file_list' is not defined

In [1]:
x_train[0]

NameError: name 'x_train' is not defined

In [ ]:
callbacks = [tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True),
             tf.keras.callbacks.ModelCheckpoint(
                 './models/model_{epoch:03d}_{val_loss:.4f}.h5',
                 save_best_only=True
             )]

def learning(x_train, y_train, model, callbacks):
    model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"])
    model.fit(x_train, y_train, batch_size=32, epochs=50, validation_split=0.2, callbacks=callbacks)
    return model

In [ ]:
trained_model = learning()

# 献立推薦モデルの性能評価

In [ ]:
test_loss, test_acc = trained_model.evaluate(x_test, y_test, verbose=0)